# Non-Negative Matrix Factorization on Module 3 Movie Ratings Dataset

## Part One

First I'll load the movie ratings data and use NMF to predict the missing ratings from the test data. Then I'll measure the RMSE and discuss what it means in the context of this analysis.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix, csr_matrix
from collections import namedtuple
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import NMF

In [2]:
# Read in data and take a quick look
MV_users = pd.read_csv('users.csv')
MV_movies = pd.read_csv('movies.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
Data = namedtuple('Data', ['users','movies','train','test'])
data = Data(MV_users, MV_movies, train, test)

print(f'train data info: {data.train.info()}')
print(data.train.head(5))
print(f'Count of movies in training data with zero rating: {len(data.train[data.train['rating'] == 0])}')

print(f'users info: {data.users.info()}')
print(data.users.head(5))

print(f'movies info: {data.movies.info()}')
print(data.movies.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700146 entries, 0 to 700145
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   uID     700146 non-null  int64
 1   mID     700146 non-null  int64
 2   rating  700146 non-null  int64
dtypes: int64(3)
memory usage: 16.0 MB
train data info: None
    uID   mID  rating
0   744  1210       5
1  3040  1584       4
2  1451  1293       5
3  5455  3176       2
4  2507  3074       5
Count of movies in training data with zero rating: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uID         6040 non-null   int64 
 1   gender      6040 non-null   object
 2   age         6040 non-null   int64 
 3   accupation  6040 non-null   int64 
 4   zip         6040 non-null   object
dtypes: int64(3), object(2)
memory usage: 236.1+ KB
users info: None
   uID ge

## Train Basic NMF Unsupervised Methods

NMF is specifically intended for datasets with non-negative values, so it seems like a reasonable choice for predicting movie ratings ranging from 1-5. It typically handles sparse matrices well. The results of NMF will also be more easily interpretable based on the constraints of a movie rating problem (compared to methods like SVD which couild predict negative values).

In [3]:
# First map users and movies to correct indices
user_map = {u: i for i, u in enumerate(data.train['uID'].unique())}
movie_map = {m: j for j, m in enumerate(data.train['mID'].unique())}
n_users = len(user_map)
n_movies = len(movie_map)

# Convert uID and mID into matrix indices
train_user_idx = data.train['uID'].map(user_map)
train_movie_idx = data.train['mID'].map(movie_map)
# Repeat for test data
test_user_idx = data.test['uID'].map(user_map)
test_movie_idx = data.test['mID'].map(movie_map)

# For test data, we want to remove blank instances where user and movie are both null
na_mask = test_user_idx.notna() & test_movie_idx.notna()
test_clean = data.test[na_mask]
test_user_idx = test_user_idx[na_mask].astype(int)
test_movie_idx = test_movie_idx[na_mask].astype(int)

# Create sparse matrix for user-movie train data
M_train = csr_matrix((data.train['rating'], (train_user_idx, train_movie_idx)), shape=(n_users, n_movies))

# BASELINE NMF MODEL
nmf_base = NMF(n_components=25, random_state=42, max_iter=300)

# fit_transform base model on sparse training matrix
# W has dim n_users x n_components
W_base = nmf_base.fit_transform(M_train)
# H has dim n_components x n_movies
H_base = nmf_base.components_

# Create predictor from train set decomposed matrices
y_pred_base = np.dot(W_base, H_base)

# Run RMSE on predictions
test_preds_base = y_pred_base[test_user_idx, test_movie_idx]

# Calculate RMSE
rmse_base = np.sqrt(mean_squared_error(test_clean['rating'], test_preds_base))
print(f'Baseline NMF Test RMSE: {rmse_base:0.4f}')

Baseline NMF Test RMSE: 2.8594


/opt/anaconda3/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 300 reached. Increase it to improve convergence.
  warnings.warn(


### Baseline Result
The baseline RMSE isn't great at 2.86. A prediction of between 1 and 5 being off by almost 3 points leads to some pretty poor performance.

Next I'll test a grid of hyperparameters to try and find the combination that produces the lowest RMSE on test data. I'm choosing to tune the following parameters with a grid search:
* **n_components** - number of latent features the model can identify (# topics)
* **beta_loss** - different types of loss functions to optimize the model
* **l1_ratio** - regularization parameter for loss function
(Source: NMF https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html)

Once I have these results, I'll use my tuned, trained NMF method to make predictions on the test data and calculate the RMSE.

In [6]:
# HYPERPARAMETER TUNING

n_components_arr = [20, 40, 60, 80]
beta_loss_arr = ['frobenius', 'kullback-leibler']

tune_results = []

for k in n_components_arr:
    for b in beta_loss_arr:        
        nmf = NMF(
            n_components=k,
            beta_loss=b,
            solver='mu',
            random_state=42,
            max_iter=300 # just picked a likely value that left the runtime manageable
        )
            
        # Fit on training matrix
        W = nmf.fit_transform(M_train)
        H = nmf.components_
            
        # Predict ratings
        R_pred = W @ H
            
        # Evaluate on test set
        preds = R_pred[test_user_idx, test_movie_idx]
        rmse = np.sqrt(mean_squared_error(test_clean['rating'], preds))
    
        tune_results.append({
                'n_components': k,
                'beta_loss': b,
                'RMSE': rmse
            })
tune_results_df = pd.DataFrame(tune_results).sort_values(by='RMSE', ascending=True)
print(tune_results_df)

   n_components         beta_loss      RMSE
3            40  kullback-leibler  2.855662
0            20         frobenius  2.867565
1            20  kullback-leibler  2.879435
5            60  kullback-leibler  2.888133
2            40         frobenius  2.920254
7            80  kullback-leibler  2.929562
4            60         frobenius  2.968313
6            80         frobenius  2.999120


## Results & Discussion

The best NMF model for this dataset still only achieves an RMSE of **2.86** on test data (using n_components=40 and a KL beta loss function).

In the Module 3 Lab, even the worst baseline recommender model (using the predict_everything_to_3 method) had an RMSE of around 1.3, which is significantly better performance than NMF on this data.

The likely reason for this is that NMF tries to make predictions for ratings between ALL users and ALL movies, when the sparse nature of the data only makes it possible for a small subset of user-movie pairs to actually provide data to train off of. NMF works best on dense matrices. In this case, the loss/optimization problem that NMF is trying to solve has limited data to learn from, so the predictions are not very good. It also assumes that there is meaningful latent structure in feature space that can be identified to help categorize the data, which may not be true in all cases.

By contrast, even though the predict_everything_to_3 strategy (alongside the other baseline recommender models) is not very complex, it is reliably able to fill in these blanks with reasonable values by using a global mean rating of 3 before continuing to learn about the item-item relationships. This ends up producing a much more robust predictive model.

**Ways to fix this**

NMF could be improved if data was normalized before training. This could include subtracting out individual user bias before matrix factorization, or subtracting 3 (to account for the same global mean as the predict_everything_to_3 method).

## Sources



* NMF

  https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html

* Step-by-Step NMF Example in Python | by Quin Daly | Medium (July 13, 2023)

  Quin Daly

  https://medium.com/@quindaly/step-by-step-nmf-example-in-python-9974e38dc9f9